## Import packages

In [1]:
# preprocessing
from DataPreprocess import *
from DataFetch import fetch_business_license, fetch_econ_indicators

# viz
import altair as alt
alt.data_transformers.enable("vegafusion")
import matplotlib.pyplot as plt

## Loading dataset

In [7]:
# Fetch data by urls --> already modulized
business = fetch_business_license()
raw_econ_index_data_dict = fetch_econ_indicators()

# It takes a while to load data from the url, so... here's the shortcut!
# Just download the file above to your local machine, and put the file in the data folder
business = pd.read_csv('data/business-licences.csv', delimiter = ';')

raw_econ_index_data_dict = {
    'GDP': pd.read_csv('data/gdp_by_industry.csv'),
    'ConsumerPrice': pd.read_csv('data/consumer_price_index.csv'),
    'Employment': pd.read_csv('data/employment_by_industry.csv'),
    'InvestmentConstruction': pd.read_csv('data/investment_in_building_construction.csv')
}

Now loading: business_license data


/Users/oybb/Desktop/UBC_MDS/block3/522_ds_workflow/dsci522_group1/DataFetch.py:11: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  Example


Now loading: GDP data
Now loading: ConsumerPrice data
Now loading: Employment data
Now loading: InvestmentConstruction data


## Preprocessing

### Business Lisence data
#### Clean-up
- Drop rows where `ExpiredDate` and `IssuedDate` are NA.
- Transform `ExpiredDate` and `IssuedDate` to date.
- Calculate the survival interval of each company, which is the difference between the maximum of ExpiredDate and the minimum of IssuedDate.
- Keep only the newest issued record of each company.
- Filter to keep those records where the latest `ExpiredDate` is before or equal to year 2022 because for those licenses issued in year 2023, the dafault `ExpiredDate` are `2023-12-31` and we cannot know whether it would survive until then.

#### Response Variable for Classification: survival_status
- To balance the amount of True & False, set the threshold to 2 years 
- Adjust Boolean value to 0, 1

In [3]:
# Drop rows where ExpiredDate and IssuedDate are NA
business = business_datacleaning(business = business, survival_threshold = 365 * 2)
business

,FOLDERYEAR,LicenceRSN,LicenceNumber,LicenceRevisionNumber,BusinessName,BusinessTradeName,Status,IssuedDate,ExpiredDate,BusinessType,...,Country,PostalCode,LocalArea,NumberofEmployees,FeePaid,ExtractDate,Geom,geo_point_2d,survival_days,survival_status
0,2013,1786043,13-166627,0,Melissa Cheryl Aston (Melissa Aston),Kazoomko Productions,Issued,2012-12-29,2013-12-31,Entertainment Services,...,CA,NaN,Kitsilano,0.0,129.0,2019-07-21T13:49:06-07:00,NaN,NaN,1828.0,1
1,2013,1786044,13-166628,0,Corus Radio Company,CHMJ AM730 and CFOX 99.3FM,Issued,2013-01-14,2013-12-31,Entertainment Services,...,CA,V7Y 1K9,Downtown,0.0,129.0,2019-07-21T13:49:06-07:00,"{""coordinates"": [-123.119500778402, 49.2822434...","49.2822434350563, -123.119500778402",2908.0,1
2,2013,1786048,13-166632,0,Jamieson Productions Inc,Jamieson Prod Inc,Issued,2013-09-12,2013-12-31,Entertainment Services,...,CA,NaN,Downtown,0.0,191.0,2019-07-21T13:49:06-07:00,NaN,NaN,1936.0,1
4,2013,1786055,13-166639,0,(Jessica Minnie),Petite Pearl Wedding and Event Planning,Issued,2013-06-17,2013-12-31,Entertainment Services,...,CA,NaN,Downtown,0.0,191.0,2019-07-21T13:49:06-07:00,NaN,NaN,3849.0,1
9,2013,1786065,13-166649,0,Holly Perrin Yoos (Holly Yoos),Copperplate Communications,Issued,2012-11-29,2013-12-31,Entertainment Services,...,CA,NaN,Kitsilano,0.0,129.0,2019-07-21T13:49:06-07:00,NaN,NaN,4049.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
643052,2021,3859186,21-260945,0,Betterbite Ltd,NaN,Issued,2021-08-16,2021-12-31,Moving/Transfer Service,...,CA,NaN,Oakridge,1.0,138.0,2023-11-01T02:38:58-07:00,NaN,NaN,867.0,1
643061,2021,3859492,21-261247,0,Vancouver Charcuterie Inc,Charcuterie Vancouver,Issued,2021-10-06,2021-12-31,Ltd Service Food Establishment,...,CA,V6K 1R1,Kitsilano,1.0,155.0,2023-11-01T02:38:58-07:00,"{""coordinates"": [-123.167908743383, 49.2680903...","49.2680903409461, -123.167908743383",816.0,1
643067,2021,3859939,21-261673,0,Ian Martin Information Technology Inc,NaN,Issued,2021-08-11,2021-12-31,Employment Agency,...,CA,V6E 3L2,Downtown,2.0,125.0,2023-11-01T02:38:58-07:00,"{""coordinates"": [-123.121522823224, 49.2870270...","49.2870270555211, -123.121522823224",142.0,0
643070,2021,3860003,21-261730,0,Glee Road Productions Ltd,NaN,Issued,2021-07-19,2021-12-31,Production Company,...,CA,V5L 1R2,Grandview-Woodland,60.0,138.0,2023-11-01T02:38:58-07:00,"{""coordinates"": [-123.06398007395, 49.28178427...","49.2817842705027, -123.06398007395",165.0,0


### Macroeconomics Data
- Create a column `REF_YEAR` representing the year of `REF_DATE`
- Keep rows where `North American Industry Classification System (NAICS) == 'All industries'`, since it is time-consuming to manually map the `BusinessType` in business license dataset to the related industries, we will merely consider the overall GDP performance in this project.
- Keep rows where `REF_YEAR >= 2012`
- Keep columns `REF_YEAR` and `VALUE`

_Delete for now: Adjust `REF_DATE` from `%y-%m` to `%y-%m-01` by applying `pd.to_datetime`_

In [4]:
econ = econ_datacleaning(raw_econ_index_data_dict)
econ

,FOLDERYEAR,GDPValue,ConsumerPriceValue,EmploymentValue,InvestmentConstructionValue
0,2012,1.710429e+06,1.758333,2296.708333,9.128106e+08
1,2013,1.754173e+06,1.266667,2320.475000,9.875339e+08
2,2014,1.803636e+06,1.475000,2348.983333,1.036283e+09
3,2015,1.820026e+06,1.891667,2390.000000,1.146144e+09
4,2016,1.839614e+06,1.708333,2468.166667,1.267791e+09
5,2017,1.901971e+06,1.225000,2560.616667,1.287776e+09
6,2018,1.958470e+06,1.800000,2607.116667,1.434572e+09
7,2019,1.996744e+06,2.166667,2676.116667,1.657493e+09
8,2020,1.897187e+06,1.633333,2509.850000,1.519591e+09
9,2021,1.991978e+06,2.566667,2665.416667,1.504690e+09


### Combine business lisence and macroeconomics data
- Map the yearly GDP value to the first lisence issued year of each company
- _Discussion: Since the threshold of survival for now is 2 years, whether or not we map the average of `GDP value of the first lisence issued year and the next year` instead of merely the first license issued year?_

In [5]:
business_econ = merge_business_econ_by_year(business, econ)

In [6]:
business_econ

,FOLDERYEAR,LicenceRSN,LicenceNumber,LicenceRevisionNumber,BusinessName,BusinessTradeName,Status,IssuedDate,ExpiredDate,BusinessType,...,FeePaid,ExtractDate,Geom,geo_point_2d,survival_days,survival_status,GDPValue,ConsumerPriceValue,EmploymentValue,InvestmentConstructionValue
0,2013,1786043,13-166627,0,Melissa Cheryl Aston (Melissa Aston),Kazoomko Productions,Issued,2012-12-29,2013-12-31,Entertainment Services,...,129.0,2019-07-21T13:49:06-07:00,NaN,NaN,1828.0,1,1.754173e+06,1.266667,2320.475000,9.875339e+08
1,2013,1786044,13-166628,0,Corus Radio Company,CHMJ AM730 and CFOX 99.3FM,Issued,2013-01-14,2013-12-31,Entertainment Services,...,129.0,2019-07-21T13:49:06-07:00,"{""coordinates"": [-123.119500778402, 49.2822434...","49.2822434350563, -123.119500778402",2908.0,1,1.754173e+06,1.266667,2320.475000,9.875339e+08
2,2013,1786048,13-166632,0,Jamieson Productions Inc,Jamieson Prod Inc,Issued,2013-09-12,2013-12-31,Entertainment Services,...,191.0,2019-07-21T13:49:06-07:00,NaN,NaN,1936.0,1,1.754173e+06,1.266667,2320.475000,9.875339e+08
3,2013,1786055,13-166639,0,(Jessica Minnie),Petite Pearl Wedding and Event Planning,Issued,2013-06-17,2013-12-31,Entertainment Services,...,191.0,2019-07-21T13:49:06-07:00,NaN,NaN,3849.0,1,1.754173e+06,1.266667,2320.475000,9.875339e+08
4,2013,1786065,13-166649,0,Holly Perrin Yoos (Holly Yoos),Copperplate Communications,Issued,2012-11-29,2013-12-31,Entertainment Services,...,129.0,2019-07-21T13:49:06-07:00,NaN,NaN,4049.0,1,1.754173e+06,1.266667,2320.475000,9.875339e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92312,2021,3859186,21-260945,0,Betterbite Ltd,NaN,Issued,2021-08-16,2021-12-31,Moving/Transfer Service,...,138.0,2023-11-01T02:38:58-07:00,NaN,NaN,867.0,1,1.991978e+06,2.566667,2665.416667,1.504690e+09
92313,2021,3859492,21-261247,0,Vancouver Charcuterie Inc,Charcuterie Vancouver,Issued,2021-10-06,2021-12-31,Ltd Service Food Establishment,...,155.0,2023-11-01T02:38:58-07:00,"{""coordinates"": [-123.167908743383, 49.2680903...","49.2680903409461, -123.167908743383",816.0,1,1.991978e+06,2.566667,2665.416667,1.504690e+09
92314,2021,3859939,21-261673,0,Ian Martin Information Technology Inc,NaN,Issued,2021-08-11,2021-12-31,Employment Agency,...,125.0,2023-11-01T02:38:58-07:00,"{""coordinates"": [-123.121522823224, 49.2870270...","49.2870270555211, -123.121522823224",142.0,0,1.991978e+06,2.566667,2665.416667,1.504690e+09
92315,2021,3860003,21-261730,0,Glee Road Productions Ltd,NaN,Issued,2021-07-19,2021-12-31,Production Company,...,138.0,2023-11-01T02:38:58-07:00,"{""coordinates"": [-123.06398007395, 49.28178427...","49.2817842705027, -123.06398007395",165.0,0,1.991978e+06,2.566667,2665.416667,1.504690e+09
